In [ ]:
import numpy as np
import pandas as pd

from datetime import datetime
from pathlib import Path

%matplotlib inline
import matplotlib.pyplot as plt

#pd.set_option("display.max_columns", 100)

DATA_PATH = Path.cwd().parent / 'data' / 'processed'
SUBMISSION_PATH = Path.cwd().parent / 'models' / 'submissions'

algo_name = 'LogReg_1'
data_name = 'employ_1hot'
date = datetime.now().strftime('%Y-%m-%d')

training_features_file_name = 'training_set_features_' + data_name + '.csv'
training_features_file = DATA_PATH / training_features_file_name
test_features_file_name = 'test_set_features_' + data_name + '.csv'
test_features_file = DATA_PATH / test_features_file_name

training_labels_file = DATA_PATH / 'training_set_labels.csv'
submission_format_file = DATA_PATH / "submission_format.csv"

submission_file_name = algo_name + '_' + data_name + '_' + date + '.csv'
submission_file = SUBMISSION_PATH / submission_file_name

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier

from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_curve, roc_auc_score

RANDOM_SEED = 6    # Set a random seed for reproducibility!

# load data
training_features_df = pd.read_csv(training_features_file, index_col="respondent_id")
training_labels_df = pd.read_csv(training_labels_file, index_col="respondent_id")

numeric_cols = training_features_df.columns[training_features_df.dtypes != "object"].values
#print(numeric_cols)

# chain preprocessing into a Pipeline object
# each step is a tuple of (name you chose, sklearn transformer)
numeric_preprocessing_steps = Pipeline([
    ('standard_scaler', StandardScaler()),
    ('simple_imputer', SimpleImputer(strategy='median'))
])

# create the preprocessor stage of final pipeline
# each entry in the transformer list is a tuple of
# (name you choose, sklearn transformer, list of columns)
preprocessor = ColumnTransformer(
    transformers = [
        ("numeric", numeric_preprocessing_steps, numeric_cols)
    ],
    remainder = "drop"
)

estimators = MultiOutputClassifier(
    estimator=LogisticRegression(penalty="l2", C=1)
)

full_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("estimators", estimators),
])

#full_pipeline

X_train, X_eval, y_train, y_eval = train_test_split(
    training_features_df,
    training_labels_df,
    test_size=0.33,
    shuffle=True,
    stratify=training_labels_df,
    random_state=RANDOM_SEED
)

#%%time

# Train model
full_pipeline.fit(X_train, y_train)

# Predict on evaluation set
# This competition wants probabilities, not labels
preds = full_pipeline.predict_proba(X_eval)
#preds

#print("test_probas[0].shape", preds[0].shape)
#print("test_probas[1].shape", preds[1].shape)

y_preds = pd.DataFrame(
    {
        "h1n1_vaccine": preds[0][:, 1],
        "seasonal_vaccine": preds[1][:, 1],
    },
    index = y_eval.index
)
#print("y_preds.shape:", y_preds.shape)
#y_preds.head()

def plot_roc(y_true, y_score, label_name, ax):
    fpr, tpr, thresholds = roc_curve(y_true, y_score)
    ax.plot(fpr, tpr)
    ax.plot([0, 1], [0, 1], color='grey', linestyle='--')
    ax.set_ylabel('TPR')
    ax.set_xlabel('FPR')
    ax.set_title(
        f"{label_name}: AUC = {roc_auc_score(y_true, y_score):.4f}"
    )

fig, ax = plt.subplots(1, 2, figsize=(7, 3.5))

plot_roc(
    y_eval['h1n1_vaccine'], 
    y_preds['h1n1_vaccine'], 
    'h1n1_vaccine',
    ax=ax[0]
)

plot_roc(
    y_eval['seasonal_vaccine'], 
    y_preds['seasonal_vaccine'], 
    'seasonal_vaccine',
    ax=ax[1]
)

fig.tight_layout()

In [ ]:
roc_auc_score(y_eval, y_preds)

In [ ]:
#%%time 

full_pipeline.fit(training_features_df, training_labels_df)

None   # So we don't print out the whole pipeline representation

test_features_df = pd.read_csv(test_features_file, index_col="respondent_id")
test_probas = full_pipeline.predict_proba(test_features_df)
#test_probas

submission_df = pd.read_csv(submission_format_file, index_col="respondent_id")
#submission_df.head()

# Make sure we have the rows in the same order
np.testing.assert_array_equal(test_features_df.index.values, submission_df.index.values)

# Save predictions to submission data frame
submission_df["h1n1_vaccine"] = test_probas[0][:, 1]
submission_df["seasonal_vaccine"] = test_probas[1][:, 1]
#submission_df.head()

submission_df.to_csv(submission_file, index=True)
print(submission_file)